In [2]:
import pandas as pd
data = pd.read_json('cladder.json')
data = data.dropna().reset_index(drop=True)

#create column "formal_form" and "graph" that holds the value corresponding to the key "formal_form" at the dict inside column "meta"

data = data.assign(formal_form = lambda x: (x['meta']))
df = data['formal_form'].apply(lambda x: (x.get("formal_form")))
data['formal_form'] = df

data = data.assign(graph = lambda x: (x['reasoning']))
df = data['graph'].apply(lambda x: (x.get("step1")))
data['graph'] = df

data = data.assign(query_type = lambda x: (x['meta']))
df = data['query_type'].apply(lambda x: (x.get("query_type")))
data['query_type'] = df

data = data.assign(info = lambda x: (x['meta']))
df = data['info'].apply(lambda x: (x.get("given_info")))
data['info'] = df
data


,question_id,desc_id,given_info,question,answer,meta,reasoning,formal_form,graph,query_type,info
0,4,alarm-mediation-nde-model0-spec0-q0,For husbands that don't set the alarm and wive...,If we disregard the mediation effect through w...,yes,"{'story_id': 'alarm', 'graph_id': 'mediation',...",{'step0': 'Let X = husband; V2 = wife; Y = ala...,"E[Y_{X=1, V2=0} - Y_{X=0, V2=0}]","X->V2,X->Y,V2->Y",nde,"{'p(Y | X, V2)': [[0.08430222457648501, 0.5394..."
1,7,alarm-mediation-ate-model1-spec1-q1,"For husbands that don't set the alarm, the pro...",Will alarm set by husband decrease the chance ...,no,"{'story_id': 'alarm', 'graph_id': 'mediation',...",{'step0': 'Let X = husband; V2 = wife; Y = ala...,E[Y | do(X = 1)] - E[Y | do(X = 0)],"X->V2,X->Y,V2->Y",ate,"{'p(Y | X)': [0.26076060984306904, 0.755829649..."
2,8,alarm-mediation-marginal-model1-spec1-q0,The overall probability of alarm set by husban...,Is ringing alarm more likely than silent alarm...,yes,"{'story_id': 'alarm', 'graph_id': 'mediation',...",{'step0': 'Let X = husband; V2 = wife; Y = ala...,P(Y),"X->V2,X->Y,V2->Y",marginal,"{'P(X)': 0.771884206668314, 'P(Y | X)': [0.260..."
3,15,alarm-mediation-ate-model3-spec3-q1,"For husbands that don't set the alarm, the pro...",Will alarm set by husband decrease the chance ...,no,"{'story_id': 'alarm', 'graph_id': 'mediation',...",{'step0': 'Let X = husband; V2 = wife; Y = ala...,E[Y | do(X = 1)] - E[Y | do(X = 0)],"X->V2,X->Y,V2->Y",ate,"{'p(Y | X)': [0.19568691380685402, 0.683664327..."
4,21,alarm-mediation-nie-model4-spec4-q0,For husbands that don't set the alarm and wive...,Does husband positively affect alarm clock thr...,no,"{'story_id': 'alarm', 'graph_id': 'mediation',...",{'step0': 'Let X = husband; V2 = wife; Y = ala...,"E[Y_{X=0, V2=1} - Y_{X=0, V2=0}]","X->V2,X->Y,V2->Y",nie,"{'p(Y | X, V2)': [[0.107877091110262, 0.602626..."
...,...,...,...,...,...,...,...,...,...,...,...
8527,31012,nonsense9-fork-det-counterfactual-model3842-sp...,We know that zuph or jyka causes glimx. We obs...,Would an individual is glimx if not zuph inste...,yes,"{'story_id': 'nonsense9', 'graph_id': 'fork', ...",{'step0': 'Let V2 = jyka; X = zuph; Y = glimx....,Y_{X=0} = 1 | V2=1,"X->Y,V2->Y",det-counterfactual,{'V2': 1}
8528,31014,nonsense9-fork-det-counterfactual-model3842-sp...,We know that zuph or jyka causes glimx. We obs...,Would an individual is glimx if zuph instead o...,yes,"{'story_id': 'nonsense9', 'graph_id': 'fork', ...",{'step0': 'Let V2 = jyka; X = zuph; Y = glimx....,Y_{X=1} = 1 | V2=1,"X->Y,V2->Y",det-counterfactual,{'V2': 1}
8529,31015,nonsense9-fork-det-counterfactual-model3842-sp...,We know that zuph or jyka causes glimx. We obs...,Would an individual is not glimx if zuph inste...,no,"{'story_id': 'nonsense9', 'graph_id': 'fork', ...",{'step0': 'Let V2 = jyka; X = zuph; Y = glimx....,Y_{X=1} = 0 | V2=1,"X->Y,V2->Y",det-counterfactual,{'V2': 1}
8530,31016,nonsense9-fork-det-counterfactual-model3843-sp...,We know that zuph and jyka causes glimx. We ob...,Would an individual is glimx if not zuph inste...,no,"{'story_id': 'nonsense9', 'graph_id': 'fork', ...",{'step0': 'Let V2 = jyka; X = zuph; Y = glimx....,Y_{X=0} = 1 | V2=0,"X->Y,V2->Y",det-counterfactual,{'V2': 0}


In [3]:
#exchange "formal_form" and "graph" column's variables to their respective "true name" (ex: exchange X to husband, V2 to wife, etc)
def exchange(x, form, aux):
    x = x.get('step0')
    if(aux == 1):
        form = form.get('formal_form')
    elif(aux == 0):
        form = form.get('step1')
    else:
        form = str(form)
    
    x = x[4:len(x)-1] #erase "Let " and the last "."
    x = x.split('; ')

    for i in range(len(x)):
        aux = x[i].split(" = ")
        form = form.replace(aux[0],aux[1])
    return form


#exchange query_type abbreviation to the whole word

def expand(abb):
    if abb == 'nde':
        return 'natural direct effect'
    elif abb == 'ate':
        return 'average treatment effect'
    elif abb == 'marginal':
        return 'marginal probability'
    elif abb == 'nie':
        return 'natural indirect effect'
    elif abb == 'det-counterfactual':
        return 'normal counterfactual question'
    elif abb == 'ett':
        return 'average treatment effect on treated'
    elif abb == 'correlation':
        return 'conditional probability'
    elif abb == 'exp_away':
        return 'explaining away effect'
    elif abb == 'collider_bias':
        return 'collider bias'
    elif abb == 'backadj':
        return 'backdoor adjustment set'
    return 0
    

data['formal_form'] = data.apply(lambda x: exchange(x['reasoning'], x['meta'],1), axis=1)
data['graph'] = data.apply(lambda x: exchange(x['reasoning'], x['reasoning'],0), axis=1)
data['info'] = data.apply(lambda x: exchange(x['reasoning'], x['info'],2), axis=1)
data['query_type'] = data.apply(lambda x: expand(x['query_type']), axis=1)

data

,question_id,desc_id,given_info,question,answer,meta,reasoning,formal_form,graph,query_type,info
0,4,alarm-mediation-nde-model0-spec0-q0,For husbands that don't set the alarm and wive...,If we disregard the mediation effect through w...,yes,"{'story_id': 'alarm', 'graph_id': 'mediation',...",{'step0': 'Let X = husband; V2 = wife; Y = ala...,"E[alarm clock_{husband=1, wife=0} - alarm cloc...","husband->wife,husband->alarm clock,wife->alarm...",natural direct effect,"{'p(alarm clock | husband, wife)': [[0.0843022..."
1,7,alarm-mediation-ate-model1-spec1-q1,"For husbands that don't set the alarm, the pro...",Will alarm set by husband decrease the chance ...,no,"{'story_id': 'alarm', 'graph_id': 'mediation',...",{'step0': 'Let X = husband; V2 = wife; Y = ala...,E[alarm clock | do(husband = 1)] - E[alarm clo...,"husband->wife,husband->alarm clock,wife->alarm...",average treatment effect,{'p(alarm clock | husband)': [0.26076060984306...
2,8,alarm-mediation-marginal-model1-spec1-q0,The overall probability of alarm set by husban...,Is ringing alarm more likely than silent alarm...,yes,"{'story_id': 'alarm', 'graph_id': 'mediation',...",{'step0': 'Let X = husband; V2 = wife; Y = ala...,P(alarm clock),"husband->wife,husband->alarm clock,wife->alarm...",marginal probability,"{'P(husband)': 0.771884206668314, 'P(alarm clo..."
3,15,alarm-mediation-ate-model3-spec3-q1,"For husbands that don't set the alarm, the pro...",Will alarm set by husband decrease the chance ...,no,"{'story_id': 'alarm', 'graph_id': 'mediation',...",{'step0': 'Let X = husband; V2 = wife; Y = ala...,E[alarm clock | do(husband = 1)] - E[alarm clo...,"husband->wife,husband->alarm clock,wife->alarm...",average treatment effect,{'p(alarm clock | husband)': [0.19568691380685...
4,21,alarm-mediation-nie-model4-spec4-q0,For husbands that don't set the alarm and wive...,Does husband positively affect alarm clock thr...,no,"{'story_id': 'alarm', 'graph_id': 'mediation',...",{'step0': 'Let X = husband; V2 = wife; Y = ala...,"E[alarm clock_{husband=0, wife=1} - alarm cloc...","husband->wife,husband->alarm clock,wife->alarm...",natural indirect effect,"{'p(alarm clock | husband, wife)': [[0.1078770..."
...,...,...,...,...,...,...,...,...,...,...,...
8527,31012,nonsense9-fork-det-counterfactual-model3842-sp...,We know that zuph or jyka causes glimx. We obs...,Would an individual is glimx if not zuph inste...,yes,"{'story_id': 'nonsense9', 'graph_id': 'fork', ...",{'step0': 'Let V2 = jyka; X = zuph; Y = glimx....,glimx_{zuph=0} = 1 | jyka=1,"zuph->glimx,jyka->glimx",normal counterfactual question,{'jyka': 1}
8528,31014,nonsense9-fork-det-counterfactual-model3842-sp...,We know that zuph or jyka causes glimx. We obs...,Would an individual is glimx if zuph instead o...,yes,"{'story_id': 'nonsense9', 'graph_id': 'fork', ...",{'step0': 'Let V2 = jyka; X = zuph; Y = glimx....,glimx_{zuph=1} = 1 | jyka=1,"zuph->glimx,jyka->glimx",normal counterfactual question,{'jyka': 1}
8529,31015,nonsense9-fork-det-counterfactual-model3842-sp...,We know that zuph or jyka causes glimx. We obs...,Would an individual is not glimx if zuph inste...,no,"{'story_id': 'nonsense9', 'graph_id': 'fork', ...",{'step0': 'Let V2 = jyka; X = zuph; Y = glimx....,glimx_{zuph=1} = 0 | jyka=1,"zuph->glimx,jyka->glimx",normal counterfactual question,{'jyka': 1}
8530,31016,nonsense9-fork-det-counterfactual-model3843-sp...,We know that zuph and jyka causes glimx. We ob...,Would an individual is glimx if not zuph inste...,no,"{'story_id': 'nonsense9', 'graph_id': 'fork', ...",{'step0': 'Let V2 = jyka; X = zuph; Y = glimx....,glimx_{zuph=0} = 1 | jyka=0,"zuph->glimx,jyka->glimx",normal counterfactual question,{'jyka': 0}


In [5]:
a['question'][100]

'If we disregard the mediation effect through occupation, would gender positively affect salary?'

In [4]:
a = data[data['query_type'] == 'natural direct effect'].reset_index(drop=True)

In [22]:
a['reasoning'][100]

{'step0': 'Let V2 = skill; X = gender; V3 = occupation; Y = salary.',
 'step1': 'X->V3,V2->V3,X->Y,V2->Y,V3->Y',
 'step2': 'E[Y_{X=1, V3=0} - Y_{X=0, V3=0}]',
 'step3': '\\sum_{V3=v} [\\sum_{V2=k}[P(Y=1|X=1,V3=v)-P(Y=1|X=0,V3=v)]*P(V3=v|X=0,V2=k)*P(V2=k)]',
 'step4': 'P(Y=1 | X=0, V3=0) = 0.37\nP(Y=1 | X=0, V3=1) = 0.17\nP(Y=1 | X=1, V3=0) = 0.62\nP(Y=1 | X=1, V3=1) = 0.41\nP(V3=1 | X=0, V2=0) = 0.49\nP(V3=1 | X=0, V2=1) = 0.22\nP(V3=1 | X=1, V2=0) = 0.77\nP(V3=1 | X=1, V2=1) = 0.49\nP(V2=1) = 0.92',
 'step5': '0.92 * (0.41 - 0.62) * 0.22 + 0.08 * (0.17 - 0.37) * 0.77 = 0.25',
 'end': '0.25 > 0'}

In [ ]:
#marginal probability
def marg_prob(P_X, P_Y_X):
    PY = 0
    for i in range(len(P_X)):
        PY += P_X[i]*P_Y_X[i]
    return PY

#natural direct effect
def NDE():